In [ ]:
import nglview as ng
import pandas as pd

In [ ]:
%cd Urea

In [ ]:
view = ng.show_structure_file("input/prot1.pdb")
view

In [ ]:
!grep -v HETATM input/prot1.pdb > prot1_protein_tmp.pdb
!grep -v CONECT prot1_protein_tmp.pdb > prot1_protein.pdb

!grep MISSING input/prot1.pdb

In [ ]:
!gmx pdb2gmx -f prot1_protein.pdb -o prot1_processed.gro -water tip4p -ff "amber03"

In [ ]:
!ls

In [ ]:
stop.

<font size="3">Before proceeding with the simulation run, make sure to include the [atomtypes] for urea in the newly generated topology. Then in addition to including your water topology, also include the urea itp file called UREA1.itp. Finally, if it should be relevant, include the position restraints for urea.</font> 

<font size="5">An example topology file exists in the input folder, called example.top</font> 

In [ ]:
!gmx editconf -f prot1_processed.gro -o prot1_newbox.gro -c -d 1.0 -bt dodecahedron

In [ ]:
!gmx solvate -cp prot1_newbox.gro -cs UREA.gro -o prot1_solv.gro -p topol.top -maxsol 11000

In [ ]:
stop.

<font size="3">Change the name of SOL to URE in the topology file.</font> 

In [ ]:
!touch ions.mdp

In [ ]:
!gmx grompp -f ions.mdp -c prot1_solv.gro -p topol.top -o ions.tpr

In [ ]:
!printf "URE\n" | gmx genion -s ions.tpr -o prot1_solv_ions.gro -conc 0.15 -p \
topol.top -pname NA -nname CL -neutral

In [ ]:
!gmx solvate -cp prot1_solv_ions.gro -cs tip4p.gro -o prot1_solv_water.gro -p topol.top

In [ ]:
!tail -6 topol.top

In [ ]:
!cat input/run-em-amber.mdp

In [ ]:
!gmx grompp -f input/run-em-amber.mdp -c prot1_solv_water.gro -p topol.top -o em.tpr

In [ ]:
!gmx mdrun -v -deffnm em 

In [ ]:
!printf "Potential\n0\n" | gmx energy -f em.edr -o potential.xvg -xvg none

In [ ]:
df = pd.read_csv('potential.xvg', sep='\s+', header=None, names=['time','energy'])
df.plot('time')

In [ ]:
!cat input/run-md-nvt-amber.mdp

In [ ]:
!gmx grompp -f input/run-md-nvt-amber.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr 
!gmx mdrun -ntmpi 1 -v -deffnm nvt

In [ ]:
!echo "Temperature" | gmx energy -f nvt.edr -o temperature.xvg -xvg none

In [ ]:
df = pd.read_csv('temperature.xvg', sep='\s+', header=None, names=['time','temperature'])
df.plot('time')

In [ ]:
!cat input/run-md-npt-amber.mdp

In [ ]:
!gmx grompp -f input/run-md-npt-amber.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
!gmx mdrun -ntmpi 1 -v -deffnm npt

In [ ]:
!echo "Pressure" | gmx energy -f npt.edr -o pressure.xvg -xvg none

In [ ]:
df = pd.read_csv('pressure.xvg', sep='\s+', header=None, names=['time','pressure'])
df.plot('time')

In [ ]:
!echo "Density" | gmx energy -f npt.edr -o density.xvg -xvg none

In [ ]:
df = pd.read_csv('density.xvg', sep='\s+', header=None, names=['time','density'])
df.plot('time')

In [ ]:
!cat input/run-md-amber.mdp

In [ ]:
!gmx grompp -f input/run-md-amber.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr

In [ ]:
!gmx mdrun -ntmpi 1 -v -deffnm md

In [ ]:
!printf "1\n1\n" | gmx trjconv -s md.tpr -f md.xtc -o md_noPBC.xtc -center -pbc mol

In [ ]:
import mdtraj as md
traj = md.load("md_noPBC.xtc", top="prot1_newbox.gro")
view = ng.show_mdtraj(traj)
view

In [ ]:
!printf "1\n" | gmx mindist -s md.tpr -f md_center.xtc -pi -od mindist.xvg 